In [1]:
import numpy as np

In [2]:
num_bread = 1000

In [3]:
# Making imaginary bread.

# Weights: 50 grams, with a standard deviation of 3 grams.
weights = 50 + 3*np.random.randn(num_bread, 1)

# Temperature: 22 degrees, with a standard deviation of 5 degrees.
temps = 22 + 5*np.random.randn(num_bread, 1)

# Volume in ml: 3ml per gram of weight, plus 4ml per degree above 22, 
# plus some noise.
volume = 3*weights + 4*(temps - 22) + 5*np.random.randn(num_bread, 1)

In [4]:
X = np.c_[weights, temps]
y = volume

In [5]:
X.shape, y.shape

((1000, 2), (1000, 1))

In [6]:
# Splitting the data into training and test sets.
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [7]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

# Here is where the model learns its optimal parameters.
model.fit(X_train, y_train)

LinearRegression()

In [8]:
model.intercept_, model.coef_

(array([-92.56198309]), array([[3.0867031 , 4.01620572]]))

O modelo final é:

$$
\hat{y} = \overset{\theta_0}{-87.71} + \overset{\theta_1}{2.96} x_1 + \overset{\theta_2}{4.06} x_2
$$

In [9]:
y_pred = model.predict(X_test)

In [10]:
from sklearn.metrics import root_mean_squared_error

root_mean_squared_error(y_test, y_pred)

np.float64(4.830684627264953)

Equivalently, I could use the *normal equation* for fitting: 

In [11]:
X_train_augmented = np.c_[np.ones((X_train.shape[0], 1)), X_train]

In [12]:
theta_opt = np.linalg.inv(X_train_augmented.T @ X_train_augmented) @ X_train_augmented.T @ y_train

In [13]:
theta_opt

array([[-92.56198309],
       [  3.0867031 ],
       [  4.01620572]])

In [14]:
model.intercept_, model.coef_

(array([-92.56198309]), array([[3.0867031 , 4.01620572]]))

Suppose the baker made a mistake and duplicated a data column:

In [40]:
X = np.c_[weights, weights, temps]
y = volume

In [41]:
X[:5,:]

array([[51.16579276, 51.16579276,  6.43990581],
       [48.91657811, 48.91657811, 24.39535638],
       [47.21221897, 47.21221897, 23.46514944],
       [46.34811531, 46.34811531, 30.11662001],
       [50.65126743, 50.65126743, 21.24312817]])

In [42]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [43]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [47]:
model.intercept_, model.coef_

(array([-92.56198309]), array([[1.54335155, 1.54335155, 4.01620572]]))

In [20]:
X_train_augmented = np.c_[np.ones((X_train.shape[0], 1)), X_train]

In [21]:
try:
    np.linalg.inv(X_train_augmented.T @ X_train_augmented) @ X_train_augmented.T @ y_train
except Exception as e:
    print(e)

Singular matrix


In [22]:
M = X_train_augmented.T @ X_train_augmented

In [23]:
np.linalg.det(M)

np.float64(0.0)

In [24]:
U, S, Vt = np.linalg.svd(X_train_augmented)

In [25]:
X_train_augmented.shape

(800, 4)

In [26]:
U.shape, S.shape, Vt.shape

((800, 800), (4,), (4, 4))

Another example

In [27]:
temps_F = 1.8*temps + 32

In [28]:
X = np.c_[weights, temps, temps_F]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
)

In [30]:
X_train_augmented = np.c_[np.ones((X_train.shape[0], 1)), X_train]

In [31]:
X_train_augmented[:5,:]

array([[ 1.        , 47.06785029, 15.89635296, 60.61343533],
       [ 1.        , 51.32100774, 16.90248961, 62.42448129],
       [ 1.        , 50.75351597, 21.82846386, 71.29123494],
       [ 1.        , 50.25263658, 18.67745222, 65.61941399],
       [ 1.        , 51.87632246, 27.00015247, 80.60027445]])

In [32]:
alpha = 1e-3 / X_train_augmented.shape[1]
M = X_train_augmented.T @ X_train_augmented + alpha*np.eye(X_train_augmented.shape[1])

In [33]:
(np.linalg.inv(M) @ M).round(2)

array([[ 1., -0., -0., -0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [-0., -0., -0.,  1.]])

In [34]:
U, S, Vt = np.linalg.svd(X_train_augmented)

In [35]:
theta = np.linalg.inv(X_train_augmented.T @ X_train_augmented \
    + alpha * np.eye(X_train_augmented.shape[1])) @ X_train_augmented.T @ y_train

In [36]:
theta

array([[-0.6066643 ],
       [ 3.08670185],
       [ 9.18868821],
       [-2.87360154]])

In [37]:
model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

In [38]:
model.intercept_, model.coef_

(array([-1.19737112e+13]),
 array([[ 3.08670310e+00, -6.73521257e+11,  3.74178476e+11]]))

In [39]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=1e-3)

model.fit(X_train, y_train)

model.intercept_, model.coef_

(array([-147.12173401]), array([[3.08670264, 0.94721831, 1.70499298]]))